# Local Testing & Image Build

Use this script to test your codes in ./artifacts/src/application.py

In [1]:
import os
import yaml

In [2]:
os.environ['WORKING_PROJECT'] = 'cctype'
os.environ['INFERENCE_BASE_DIR'] = os.path.abspath('.')
with open("global-config.yml", 'r') as f:
    global_settings = yaml.safe_load(f)
    os.environ['MLFLOW_TRACKING_URL'] = global_settings['mlflow_url']

In [3]:
os.environ['MLFLOW_TRACKING_URL']

'http://localhost:5000'

## Unit testing to validate inferencing script

In [4]:
# basic unit testing (Ray is not used)
#!pytest tests/test_application.py

## Save and log model artifacts using mlflow service

In [8]:
from ecsclient.buildtools.infer import InferenceBuild

In [9]:
# connect to mlflow
inference_build = InferenceBuild()

Connecting to mlflow server: http://localhost:5000
Mlflow server connected
Connection timeout; failed to connect to mlflow server.


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [ ]:
import logging
logging.getLogger("mlflow").setLevel(logging.DEBUG)

In [ ]:
# save artifacts locally
inference_build.save_model()

In [ ]:
#local Ray based unit testing
os.environ["MODEL_DIR"]=os.path.join(os.environ['INFERENCE_BASE_DIR'],"projects", os.environ['WORKING_PROJECT'],"mlflow_model")

In [ ]:
!pytest -s tests/test_batch_inference_job.py

## Make Predictions manually

In [ ]:
# define the file location of your input data and the destination folder for the prediction results
source_path=f'/Users/ycao/codes/ecs-ml-inference-framework/projects/{os.environ["WORKING_PROJECT"]}/tests/test_data.csv'
source_format='csv'
result_path=f'/Users/ycao/codes/ecs-ml-inference-framework/projects/{os.environ["WORKING_PROJECT"]}/tests'
result_format='csv'

### Use Ray

In [ ]:
!python -m ecs.inference.jobmanager --sourcepath {source_path} --resultpath {result_path} --sourceformat {source_format} --resultformat {result_format} --suppresswarnings

### Use Regular Python

In [ ]:
import os
import sys
import pandas as pd
from ecs.utils import load_manifest
from mlflow.pyfunc import PythonModelContext

FILE_READERS = {
    'csv': pd.read_csv,
    'parquet': pd.read_parquet
}

PROJECT_DIR = f"{os.environ['INFERENCE_BASE_DIR']}/projects/{os.environ['WORKING_PROJECT']}"
mf = load_manifest(f'{PROJECT_DIR}/manifest.json')
custom_code_path = f'{PROJECT_DIR}/src'
if os.path.exists(custom_code_path):
    sys.path.append(custom_code_path)

from application import Application
app = Application()
artifacts={'manifest':f'{PROJECT_DIR}/manifest.json', 'models': f'{PROJECT_DIR}/models'}
context = PythonModelContext(artifacts)
app.load_context(context)
data = FILE_READERS[source_format](source_path)
result_data = app.predict(None, data)
save_method = getattr(result_data, f'to_{result_format}')
save_method(f'{result_path}/prediction_results.{result_format}', index=False)
os.chdir(os.environ['INFERENCE_BASE_DIR'])

### log and register model

In [ ]:
inference_build.log_model(register = True)

## Prepare Dockerfile & Build Image

In [ ]:
inference_build.prepare_image()
# modellist = inference_build.list_registered_models()

In [ ]:
inference_build.build_image()

## Local Image Unit Testing & Vulnerability Scan

In [ ]:
!docker build -f base-image\\Dockerfile_ray_inference -t ray_inference .

In [ ]:
# unit testing
!docker run ml-inference-{os.environ['WORKING_PROJECT']} pytest

In [ ]:
#vulerability scan
!docker run -v /var/run/docker.sock:/var/run/docker.sock aquasec/trivy image --security-checks vuln --timeout 15m ml-inference-{os.environ['WORKING_PROJECT']}

In [ ]:
!pip uninstall grpcio -y

In [ ]:
!conda install grpcio -y